In [ ]:
import json
import re

import pandas as pd

In [ ]:
raw_esif = pd.read_excel('input/ESF-Call-1-Successful-applications-by-Constituency-and-Council.XLS', skiprows=3)
raw_esif.shape

In [ ]:
raw_esif.columns

In [ ]:
# Just look at the columns we're interested in, rename them and remove null rows. Also clean Org name
ni_funding_call1_2014 = raw_esif[[
    'ORGANISATION NAME', 'PROJECT TITLE', 'TOTAL ELIGIBLE PROJECT COST\n',
    'ESF SOUGHT\n', 'PRIORITY / THEMATIC OBJECTIVE'
]]
ni_funding_call1_2014 = ni_funding_call1_2014.rename(
    index=str, columns={
        'TOTAL ELIGIBLE PROJECT COST\n': 'project_cost', 
        'ESF SOUGHT\n': 'eu_investment', 
        'ORGANISATION NAME': 'beneficiary',
        'PRIORITY / THEMATIC OBJECTIVE': 'investment_priority', 
        'PROJECT TITLE': 'project'
    })
ni_funding_call1_2014 = ni_funding_call1_2014[~ni_funding_call1_2014['beneficiary'].isnull()]
ni_funding_call1_2014['beneficiary'] = ni_funding_call1_2014['beneficiary'].\
    str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").\
    str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").\
    str.strip().str.upper()
ni_funding_call1_2014.head()

In [ ]:
ni_funding_call2_2014 = pd.read_excel('input/ESF-Call-2-successful-applications-by-constituency-and-council-area.xls', skiprows=2)
ni_funding_call2_2014.head()

In [ ]:
ni_funding_call2_2014.columns

In [ ]:
# Just look at the columns we're interested in and rename them. Also clean Org name
ni_funding_call2_2014 = ni_funding_call2_2014[[
    'Organisation Name', 'Project Title', 'Investment Priority',
    'Total Eligible Project Cost', 'Programme Contribution (EU and DfE) - 65%'
]]
ni_funding_call2_2014 = ni_funding_call2_2014.rename(
    index=str, columns={
        'Programme Contribution (EU and DfE) - 65%': 'eu_investment',
        'Organisation Name': 'beneficiary',
        'Project Title': 'project',
        'Investment Priority': 'investment_priority',
        'Total Eligible Project Cost': 'project_cost'
    })
ni_funding_call2_2014['beneficiary'] = ni_funding_call2_2014['beneficiary'].\
    str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").\
    str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").\
    str.strip().str.upper()
ni_funding_call2_2014.head()

In [ ]:
# Concatenate the data for call 1 and call 2 
ni_funding_2014 = pd.concat([ni_funding_call2_2014, ni_funding_call1_2014], sort=True)
ni_funding_2014.sort_values(axis=0, by='beneficiary').head()

In [ ]:
# Manually find locations for all NI organisations
ni_locations_lookup = pd.DataFrame([
    ('ACCEPTABLE ENTERPRISES', 'BT403AW'),
    ('ACCEPTABLE ENTERPRISES LTD', 'BT403AW'),       
    ('ACCESS CENTRE LTD', 'BT152GG'),
    ('ACCESS CENTRE NI', 'BT152GG'),
    ('ACTION DEAF YOUTH', 'BT42LS'),
    ('ACTION MENTAL HEALTH', 'BT234YH'),
    ('ACTION ON HEARING LOSS', 'BT28GA'),
    ('ACTION ON HEARING LOSS (FORMERLY RNID)', 'BT28GA'),
    ('ACTIVE COMMUNITIES NETWORK LIMITED', 'BT132JF'),
    ('APPLEBY CAREERS PROJECT LTD', 'BT617AE'),
    ('ASHTON COMMUNITY TRUST', 'BT152BP'),
    ('ASSOCIATION FOR REAL CHANGE NI', 'BT126TA'),
    ('BARNARDOS', 'BT43HE'),
    ("BARNARDO’S", 'BT43HE'),
    ('BRYSON CHARITABLE GROUP', 'BT27FE'),
    ('BELFAST METROPOLITAN COLLEGE', 'BT1 1HS'),
    ('CLANRYE GROUP', 'BT358SW'),
    ('CLANRYE GROUP LIMITED', 'BT358SW'),
    ('COMPASS ADVOCACY NETWORK', 'BT536JR'),
    ('COMPASS ADVOCACY NETWORK (CAN)', 'BT536JR'),                           
    ('CUSTOMISED TRAINING SERVICES', 'BT828AS'), 
    ('CUSTOMIZED TRAINING SERVICES', 'BT828AS'),
    ('CUSTOMIZED TRAINING SERVICES LTD', 'BT828AS'),
    ('DERRY & STRABANE DISTRICT COUNCIL', 'BT487NN'),                           
    ('DERRY YOUTH & COMMUNITY WORKSHOP LIMITED', 'BT486PJ'),                           
    ('DERRY YOUTH AND COMMUNITY WORKSHOP LTD', 'BT486PJ'),
    ('DFPF LTD T/A PEOPLE 1ST', 'BT96SP'),                           
    ('DISABILITY ACTION', 'BT39ED'),                           
    ('ENTERPRISE NORTHERN IRELAND', 'BT490HE'),                           
    ('ENTERPRISE NORTHERN IRELAND LTD', 'BT490HE'),                           
    ('EXTERN GROUP', 'BT364PE'),                           
    ('EXTERN N.I.', 'BT364PE'),                            
    ('EXTERN NORTHERN IRELAND', 'BT364PE'),                            
    ('FERMANAGH & OMAGH DISTRICT COUNCIL (FODC)', 'BT797BL'),                            
    ('FERMANAGH AND OMAGH DISTRICT COUNCIL', 'BT797BL'), 
    ('FIRST STEPS WOMEN CENTRE (FSWC)', 'BT701DX'),                            
    ('FIRST STEPS WOMEN\'S CENTRE', 'BT701DX'),                            
    ('GEMS NORTHERN IRELAND LTD', 'BT27DB'), 
    ('GEMS NI LTD', 'BT27DB'),
    ('GREENLIGHT GATEWAY', 'BT546EZ'),                            
    ('INCLUDE YOUTH', 'BT11QA'),                            
    ('JOB DIRECTIONS LTD', 'BT828AR'),                            
    ('KILCOOLEY WOMENS CENTRE', 'BT191QS'),
    ('LIMAVADY COMMUNITY DEVELOPMENT INITIATIVE (LCDI)', 'BT490AQ'),                           
    ('MENCAP', 'BT86BT'),                          
    ('NETWORK PERSONNEL', 'BT455AJ'),                           
    ('NETWORK PERSONNEL (NP)', 'BT455AJ'),                           
    ('NETWORK PERSONNEL LTD', 'BT455AJ'),                           
    ('NETWORK PERSONNEL LTD.', 'BT455AJ'),                           
    ('NIACRO', 'BT27GS'),                                       
    ('NOW GROUP', 'BT124GN'),
    ('NOW PROJECT LTD T/A NOW GROUP', 'BT124GN'), 
    ('ORCHARDVILLE SOCIETY LTD', 'BT6 8ED'),
    ('PEOPLEPLUS (WORKS) NI LIMITED', 'BT490LP'), 
    ('Poleglass Community Association @ Sally Gardens (PCA)', 'BT170UJ'),
    ('POLEGLASS COMMUNITY ASSOCIATION @ SALLY GARDENS (PCA)', 'BT170UJ'),
    ('RAPID', 'BT96EU'),                            
    ('RAPID LTD', 'BT96EU'),                             
    ('RCD/4RS', 'BT480LU'),                             
    ('RCD/4RS REUSE WORKSHOP', 'BT480LU'), 
    ('REED IN PARTNERSHIP', 'BT323HA'),                             
    ('RNIB', 'BT14LS'),                             
    ('RNIB NORTHERN IRELAND', 'BT14LS'),                             
    ('ROE VALLEY COMMUNITY EDUCATION FORUM LTD', 'BT499BG'),                             
    ('SHANKILL WOMEN\'S CENTRE', 'BT131FD'),                             
    ('SOMETHING SPECIAL', 'BT41NT'),   
    ('SOUTH WEST COLLEGE', 'BT808DN'),                               
    ('SOUTH WEST COLLEGE (SWC)', 'BT808DN'),                               
    ('SOUTHERN REGIONAL COLLEGE', 'BT617HF'),                               
    ('SPECIALISTERNE NI C.I.C', 'BT41AF'),                               
    ('SPRINGBOARD OPPORTUNITIES LIMITED', 'BT12GX'),                               
    ('SPRINGBOARD OPPORTUNITIES LTD', 'BT12GX'),                               
    ('START360', 'BT11PR'),                               
    ('STEPPING STONES NI', 'BT274AA'),                               
    ('THE ADVANTAGE FOUNDATION LTD', 'BT388GX'),                               
    ('THE APPLEBY TRUST', 'BT617AE'),   
    ('THE CEDAR FOUNDATION', 'BT97AS'),                              
    ('THE CONSERVATION VOLUNTEERS', 'BT60BP'),                              
    ('THE CONSERVATION VOLUNTEERS (TCV)', 'BT60BP'),                              
    ('THE ORCHARDVILLE SOCIETY', 'BT68ED'),                              
    ('THE PRINCES TRUST', 'BT125GH'),  
    ("THE PRINCE’S TRUST", 'BT125GH'),
    ('THE WOMEN\'S CENTRE DERRY', 'BT486BB'),                              
    ('TRAINING FOR WOMEN NETWORK LTD', 'BT41NY'),         
    ('TRIANGLE HOUSING ASSOCIATION LTD', 'BT536BD'),                              
    ('TRIAX', 'BT489LH'),                              
    ('TYRONE DONEGAL PARTNERSHIP', 'BT785LU'),                              
    ('ULSTER SUPPORTED EMPLOYMENT LIMITED', 'BT133JH'),                              
    ('UPPER SPRINGFIELD DEVELOPMENT TRUST', 'BT127FP'),                              
    ('UPPER SPRINGFIELD DEVELOPMENT TRUST (USDT)', 'BT127FP'),                              
    ('USEL', 'BT133JH'),                              
    ('WADE TRAINING LTD', 'BT621BA'),                              
    ('WOMEN IN BUSINESS NI', 'BT27AQ'),                              
    ('WOMEN\'S TEC', 'BT155EH'),                              
    ('WOMEN’S CENTRE DERRY', 'BT486BB'),                              
    ('WORKFORCE TRAINING SERVICES', 'BT127AJ'),                              
    ('YOUTH ACTION NORTHERN IRELAND', 'BT16AS'),                              
    ('YOUTHACTION NORTHERN IRELAND LTD', 'BT16AS')],    
columns=['beneficiary', 'raw_postcode'])
ni_locations_lookup.head()

CUSTOMISED TRAINING SERVICES has more locations. So do DISABILITY ACTION, FERMANAGH AND OMAGH DISTRICT COUNCIL, INCLUDE YOUTH, MENCAP, PEOPLEPLUS (WORKS) NI LIMITED	THE CONSERVATION VOLUNTEERS	

In [ ]:
esif = pd.merge(ni_funding_2014, ni_locations_lookup, how='left', on='beneficiary')
esif.shape

### Beneficiary

In [ ]:
esif.beneficiary[esif.beneficiary.str.strip() != esif.beneficiary]

### Project

In the absence of any IDs, do we have any duplicates? 

All of the duplicates we have look like different amounts for different years

In [ ]:
esif.project[esif.project.str.contains('\n')]

In [ ]:
esif.project[esif.project.str.strip() != esif.project] # lots
esif.project = esif.project.str.strip()

In [ ]:
esif.project.unique().shape

In [ ]:
esif[esif.duplicated(['beneficiary', 'project'], keep=False)].sort_values('project')

### Project Cost


In [ ]:
esif.project_cost.isna().sum()

In [ ]:
esif.project_cost = esif.project_cost.map(str).str.strip()
project_cost_bad = esif.project_cost.str.match(re.compile(r'.*[^0-9.].*'))
esif.project_cost[project_cost_bad]

In [ ]:
esif.project_cost = esif.project_cost.astype('float')

### EU Investment


In [ ]:
esif.eu_investment.isna().sum()

In [ ]:
esif.eu_investment = esif.eu_investment.map(str).str.strip()
eu_investment_bad = esif.eu_investment.str.match(re.compile(r'.*[^0-9.].*'))
esif.eu_investment[eu_investment_bad]

In [ ]:
esif.eu_investment = esif.eu_investment.astype('float')

### Overfunding


In [ ]:
overfunded = esif.eu_investment > esif.project_cost
esif[overfunded]

### Prop EU Financed


In [ ]:
esif['actual_prop'] = esif.eu_investment / esif.project_cost
esif.actual_prop.describe()

### Postcode


In [ ]:
[esif.shape, esif.raw_postcode.isna().sum()]

In [ ]:
ukpostcodes = pd.read_csv('../postcodes/input/ukpostcodes.csv.gz')
ukpostcodes.shape

In [ ]:
esif.raw_postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
esif['postcode'] = esif.raw_postcode.\
    str.upper().\
    str.strip().\
    str.replace(r'[^A-Z0-9]', '').\
    str.replace(r'^(\S+)([0-9][A-Z]{2})$', r'\1 \2')

In [ ]:
esif.postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
esif.postcode[~esif.postcode.isin(ukpostcodes.postcode)].unique()

## Save Data

In [ ]:
esif.columns

In [ ]:
clean_esif = esif.drop([
    'investment_priority', 'actual_prop', 'raw_postcode'
], axis=1)
clean_esif['funds'] = 'ESF'
clean_esif['start_date'] = pd.to_datetime('2014-01-01')
clean_esif['end_date'] = pd.to_datetime('2021-01-01')
clean_esif['summary'] = None
clean_esif.head()

In [ ]:
clean_esif['my_eu_id'] = clean_esif.funds.str.lower() + '_ni_' + clean_esif.index.map(str)
clean_esif.my_eu_id.head()

In [ ]:
clean_esif.to_pickle('output/esif_ni_2014_2020.pkl.gz')

## Save Map Data

In [ ]:
clean_esif_locations = pd.merge(clean_esif, ukpostcodes, validate='m:1')
clean_esif_locations.head()

In [ ]:
def make_esif_data_geo_json(data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['beneficiary', 'project', 'project_cost', 'eu_investment']
        }
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            'properties': properties
        }
    features = list(data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('output/ni_data.geo.json', 'w') as file:
    json.dump(make_esif_data_geo_json(clean_esif_locations), file, sort_keys=True)